In [ ]:
from cellpaint_database.database_loader import DatabaseLoader
import matplotlib.pyplot as plt
import tifffile
import subprocess
import os
import time

# Instructions
- Put the sqlite file into the /data folder of this repo

In [ ]:
db_path = "/Users/thomasathey/Documents/shavit-lab/fraenkel/data/aneesh/aneesh_diMNs.sqlite"  # path to sqlite file
data_root = "/Users/thomasathey/Documents/shavit-lab/fraenkel/data/aneesh"  # path to the directory from which the relative paths in the database start. In this case, the fodler that holds /images/

In [ ]:
dbl = DatabaseLoader(db_path, data_root)

# Identify Image

In [ ]:
plate = 2
well = ("B", 2, 1)  # ('row', column, field)
well2 = 1  # alternative way to specify well - site number
channel = 1

# Retrieve Image

In [ ]:
line = dbl.get_line(plate, well, channel)
treatment = dbl.get_treatment(plate, well, channel)
img = dbl.get_image(plate, well2, channel)

In [ ]:
plt.imshow(img)
plt.title(f"{line} x {treatment}")

## Retrieve Image Crop

In [ ]:
loc = (1000, 170)

line = dbl.get_line(plate, well, channel)
treatment = dbl.get_treatment(plate, well, channel)
img = dbl.get_image_loc(plate, well2, channel, loc)

In [ ]:
plt.imshow(img)
plt.title(f"{line} x {treatment} @ {loc}")

In [ ]:
rad = 30

img = dbl.get_image_loc(plate, well2, channel, loc, rad=rad)

In [ ]:
plt.imshow(img)
plt.title(f"{line} x {treatment} @ {loc} +/- {rad}")

# Open in Fiji

In [ ]:
fiji_path = "/Applications/Fiji.app/Contents/MacOS/ImageJ-macosx"  # macOS
# fiji_path = "ImageJ-win64.exe"  # Windows (assuming added to PATH)
# fiji_path = "/path/to/Fiji.app/ImageJ-linux64"  # Linux

In [ ]:
# enter lists of plates, wells, and channels
plates = [2, 2]
wells = [("B", 2, 1), ("B", 2, 1)]
channels = [1, 2]

macro = ""
temp_files = []

for i, (plate, well, channel) in enumerate(zip(plates, wells, channels)):
    line = dbl.get_line(plate, well, channel)
    treatment = dbl.get_treatment(plate, well, channel)
    img = dbl.get_image(plate, well2, channel)

    # Save as a temporary TIFF file
    temp_file = os.path.join(os.getcwd(), f"fiji_image_{i}.tif")
    tifffile.imwrite(temp_file, img)

    title = f"({plate,well,channel}): {line} x {treatment}"
    macro += f'open("{temp_file}"); rename("{title}");\n'
    temp_files.append(temp_file)

macro_file = "rename_macro.ijm"
with open(macro_file, "w") as f:
    f.write(macro)

# Launch Fiji with the macro
subprocess.Popen([fiji_path, "-macro", macro_file])

# Optional: Clean up the temporary file after viewing
time.sleep(10)
for temp_file in temp_files:
    os.remove(temp_file)
os.remove(macro_file)